# Simulation & Visualisation Examples 

You will need the libraries [ase](https://wiki.fysik.dtu.dk/ase/) and [nglview](https://github.com/nglviewer/nglview)

e.g.: `pip install ase nglview`

### Visualisation of a Protein

https://www.rcsb.org/

In [73]:
import nglview
view = nglview.show_pdbid("3pqr")  # load "3pqr" from RCSB PDB
view

NGLWidget()

In [66]:
def show_system(atoms):
    v = view(atoms, viewer="ngl")
    v.view._remote_call("setSize", target="Widget", args=["700px","700px"])
    v.view.add_ball_and_stick()
    v.view.background="black"
    return v

###  Molecular Dynamics Simulation of Acetonitrile

In [71]:
%%time
# load the necessary functions and modules
import ase.units as units
from ase.calculators.acn import ACN
from ase.constraints import FixLinearTriatomic
from ase.io import Trajectory,write
from ase.md import Langevin

# read coorrdinates from file
atoms = read("acn.xyz") 

# set up some constraints, to accelerate dynamics
atoms.constraints = FixLinearTriatomic(
    triples=[(3 * i, 3 * i + 1, 3 * i + 2)
             for i in range(nm)])

# define the molecular dynamics physics / rules / model
atoms.calc = ACN(rc=np.min(np.diag(atoms.cell)) / 2)

# define the simulation details, temperature, time steps, parameters, logfile name
md = Langevin(atoms, 1 * units.fs,
              temperature=300 * units.kB,
              friction=0.01,
              logfile=tag + '.log')

# define a file to save the generated structures during the dynamics
tag = 'acn'
traj = Trajectory(tag + '.traj', 'w', atoms)
md.attach(traj.write, interval=10)

# start the simulation for n time steps
md.run(20000)

CPU times: total: 9min 8s
Wall time: 22min 10s


True

In [72]:
# visualization of the dynamics
traj = Trajectory("acn.traj")
show_system(traj)